# Project Goal:

PREDICT percent change in stock price after next earnings event for all company

One record per earning event date per company.


# Packages Imports

In [1]:
!pip install pandas_market_calendars

# Loading Data

## Filter Trading Dates for tickers

Based on tickers in ticker csv, only keep earnings dates records for those stocks

In [2]:
import pandas as pd
import pandas_market_calendars as mcal
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import yfinance as yf

In [3]:
tickers_df = pd.read_csv("/users/brigitteasullivan/Documents/0. Data Science/Notebooks/Riipen Project/eps-stock-market-ml-project/data/all_tickers-with-options.csv", 
                         header = 0, names = ['Ticker'])
tickers_df

,Ticker
0,\t\t1INVH
1,\t\t1LNCO
2,\t\t1LUV
3,\t\t2LW
4,\t\t9AC
...,...
12829,\t\tZZ
12830,\t\tZZK
12831,\t\tZZV
12832,\t\tZZZ


In [4]:
#trim tab
tickers_df["Ticker"] = tickers_df['Ticker'].str.strip()
tickers_df

,Ticker
0,1INVH
1,1LNCO
2,1LUV
3,2LW
4,9AC
...,...
12829,ZZ
12830,ZZK
12831,ZZV
12832,ZZZ


### Earnings dates

If "WHEN" is premarket - earnings date to watch is same day 
if when is post market - earnings date to watch is next trading date. 

From documentation: https://www.earningscalendar.net/documentation

*   `date` ISO date of this earnings event.
*   `ticker` Ticker as traded on the exchange.
*   `url` URL to the press release that confirmed this event.
*   `title` Title of the press release that confirmed this event.
*   `security\_name` Name of the security trading under this symbol/ticker.
*   `exchange` The exchange where the security is trading. One of AMEX, NASDAQ, NYSE.
*   `when` When are earnings slated to be released that day. Returned only if known, otherwise the string "null".
*   `pub\_date` Publication date of the press release that confirmed this event



In [5]:
earnings_df = pd.read_csv('/users/brigitteasullivan/Documents/0. Data Science/Notebooks/Riipen Project/eps-stock-market-ml-project/data/confirmed_earnings.csv')
earnings_df

,date,exchange,symbol,when,title,url,pub_date,security_name
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDs to Announce Fiscal First Quarter 2023...,https://www.businesswire.com/news/home/2022122...,2022-12-27T11:05:00,SemiLEDS Corporation - Common Stock
1,2023-01-03,NASDAQ,SGH,postmarket,SGH Announces First Quarter Fiscal 2023 Financ...,https://www.businesswire.com/news/home/2022121...,2022-12-13T21:05:00,"SMART Global Holdings, Inc. - Ordinary Shares"
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. to Announce Fiscal ...",https://www.businesswire.com/news/home/2022122...,2022-12-21T21:05:00,"Resources Connection, Inc. - Common Stock"
3,2023-01-04,NASDAQ,SLP,postmarket,Simulations Plus Sets Date for First Quarter F...,https://www.businesswire.com/news/home/2022121...,2022-12-19T21:06:00,"Simulations Plus, Inc. - Common Stock"
4,2023-01-05,NYSE,CAG,NaN,CONAGRA BRANDS ANNOUNCES DETAILS OF FISCAL 202...,https://www.prnewswire.com/news-releases/conag...,2022-12-05T07:30:00,"ConAgra Brands, Inc. Common Stock"
...,...,...,...,...,...,...,...,...
9471,2023-12-21,NASDAQ,LMNR,postmarket,Limoneira to Announce Fourth Quarter and Full ...,https://www.businesswire.com/news/home/2023120...,2023-12-07T13:30:00,Limoneira Co - Common Stock
9472,2023-12-21,NYSE,NKE,NaN,"NIKE, Inc. Announces Second Quarter Fiscal 202...",https://www.businesswire.com/news/home/2023112...,2023-11-21T21:15:00,"Nike, Inc. Common Stock"
9473,2023-12-21,NASDAQ,PAYX,premarket,"Paychex, Inc. Schedules Second Quarter Fiscal ...",https://www.businesswire.com/news/home/2023120...,2023-12-08T19:00:00,"Paychex, Inc. - Common Stock"
9474,2023-12-21,NYSE,WS,postmarket,Worthington Steel to Webcast Discussion of Sec...,https://www.businesswire.com/news/home/2023120...,2023-12-04T21:10:00,"Worthington Steel, Inc. Common Shares"


#### Earnings

In [6]:
earnings_df = earnings_df.loc[:,['date', 'exchange', 'symbol', 'when', 'security_name']].copy()

In [7]:
earnings_df

,date,exchange,symbol,when,security_name
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDS Corporation - Common Stock
1,2023-01-03,NASDAQ,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares"
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. - Common Stock"
3,2023-01-04,NASDAQ,SLP,postmarket,"Simulations Plus, Inc. - Common Stock"
4,2023-01-05,NYSE,CAG,NaN,"ConAgra Brands, Inc. Common Stock"
...,...,...,...,...,...
9471,2023-12-21,NASDAQ,LMNR,postmarket,Limoneira Co - Common Stock
9472,2023-12-21,NYSE,NKE,NaN,"Nike, Inc. Common Stock"
9473,2023-12-21,NASDAQ,PAYX,premarket,"Paychex, Inc. - Common Stock"
9474,2023-12-21,NYSE,WS,postmarket,"Worthington Steel, Inc. Common Shares"


In [8]:
# cleaning - remove nans:
# fill missing NA's 
earnings_df.loc[:,'when'].fillna(value='premarket', inplace=True)
earnings_df

,date,exchange,symbol,when,security_name
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDS Corporation - Common Stock
1,2023-01-03,NASDAQ,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares"
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. - Common Stock"
3,2023-01-04,NASDAQ,SLP,postmarket,"Simulations Plus, Inc. - Common Stock"
4,2023-01-05,NYSE,CAG,premarket,"ConAgra Brands, Inc. Common Stock"
...,...,...,...,...,...
9471,2023-12-21,NASDAQ,LMNR,postmarket,Limoneira Co - Common Stock
9472,2023-12-21,NYSE,NKE,premarket,"Nike, Inc. Common Stock"
9473,2023-12-21,NASDAQ,PAYX,premarket,"Paychex, Inc. - Common Stock"
9474,2023-12-21,NYSE,WS,postmarket,"Worthington Steel, Inc. Common Shares"


In [10]:
# Convert the 'date' column to datetime
earnings_df['date'] = pd.to_datetime(earnings_df['date'])

In [11]:
earnings_df.dtypes

date             datetime64[ns]
exchange                 object
symbol                   object
when                     object
security_name            object
dtype: object

In [12]:
# remove data for companies not in ticker list
earnings_df = earnings_df[earnings_df['symbol'].isin(tickers_df['Ticker'])].copy()
earnings_df

,date,exchange,symbol,when,security_name
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDS Corporation - Common Stock
1,2023-01-03,NASDAQ,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares"
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. - Common Stock"
3,2023-01-04,NASDAQ,SLP,postmarket,"Simulations Plus, Inc. - Common Stock"
4,2023-01-05,NYSE,CAG,premarket,"ConAgra Brands, Inc. Common Stock"
...,...,...,...,...,...
9470,2023-12-21,NYSE,KMX,premarket,CarMax Inc
9471,2023-12-21,NASDAQ,LMNR,postmarket,Limoneira Co - Common Stock
9472,2023-12-21,NYSE,NKE,premarket,"Nike, Inc. Common Stock"
9473,2023-12-21,NASDAQ,PAYX,premarket,"Paychex, Inc. - Common Stock"


In [13]:
ticker_unique_num = tickers_df['Ticker'].nunique()
print('unique ticker symbols in ticker df:', ticker_unique_num)

symbol_unique_num = earnings_df['symbol'].nunique()
print('unique symbols in earnings df:', symbol_unique_num)


# in both ticker and earnings df
combined_df = pd.concat([earnings_df['symbol'], tickers_df['Ticker']])
unique_ticker = set(tickers_df['Ticker'].unique())
unique_earnings = set(earnings_df['symbol'].unique())

intersection = list(unique_ticker.intersection(unique_earnings))
length_intersection = len(intersection)
print('unique symbols in earnings df AND in ticker:', length_intersection)

unique ticker symbols in ticker df: 12834
unique symbols in earnings df: 2502
unique symbols in earnings df AND in ticker: 2502


In [14]:
earnings_df['exchange'].value_counts()

exchange
NYSE      4691
NASDAQ    3698
AMEX        67
OTC          2
Name: count, dtype: int64

In [15]:
# check missing values for each column
percent_missing = earnings_df.isnull().sum() * 100 / len(earnings_df)
missing_value_df = pd.DataFrame({'column_name': earnings_df.columns,
                                 'percent_missing': percent_missing})

In [16]:
missing_value_df

,column_name,percent_missing
date,date,0.000000
exchange,exchange,3.755121
symbol,symbol,0.000000
when,when,0.000000
security_name,security_name,3.709604


In [17]:
## cleaning - filter out OTC

earnings_df = earnings_df.loc[~earnings_df['exchange'].isin(["OTC"]),:]
# earnings_df

In [18]:
## QA - check that OTC not in Dataframe anymore
earnings_df['exchange'].value_counts(dropna = False)

exchange
NYSE      4691
NASDAQ    3698
NaN        330
AMEX        67
Name: count, dtype: int64

In [19]:
earnings_df

,date,exchange,symbol,when,security_name
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDS Corporation - Common Stock
1,2023-01-03,NASDAQ,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares"
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. - Common Stock"
3,2023-01-04,NASDAQ,SLP,postmarket,"Simulations Plus, Inc. - Common Stock"
4,2023-01-05,NYSE,CAG,premarket,"ConAgra Brands, Inc. Common Stock"
...,...,...,...,...,...
9470,2023-12-21,NYSE,KMX,premarket,CarMax Inc
9471,2023-12-21,NASDAQ,LMNR,postmarket,Limoneira Co - Common Stock
9472,2023-12-21,NYSE,NKE,premarket,"Nike, Inc. Common Stock"
9473,2023-12-21,NASDAQ,PAYX,premarket,"Paychex, Inc. - Common Stock"


### Event Dates (Earnings Release)

Find the next trading date based on earnings date release. 


Note:
AMEX stock exchange is also known as the NYSE American and follows the same open schedule as the NYSE, Will use the NYSE calendar for all AMEX values.

In [20]:
# dictionary of stock exchanges and codes 

stock_ex_codes = {
    'NYSE': 'XNYS',
    'New York Stock Exchange (NYSE)': 'XNYS',
    'NASDAQ': 'NASDAQ',
    'London Stock Exchange (LSE)': 'XLON',
    'Tokyo Stock Exchange (TSE)': 'XTKS',
    'Hong Kong Stock Exchange (HKEX)': 'XHKG',
    'Toronto Stock Exchange (TSX)': 'XTSE',
    'Shanghai Stock Exchange (SSE)': 'XSHG',
    'Shenzhen Stock Exchange (SZSE)': 'XSHE',
    'Bombay Stock Exchange (BSE)': 'XBOM',
    'National Stock Exchange of India (NSE)': 'XNSE',
    'AMEX': 'XNYS' # See note
}


In [21]:
# get the exchange code used to pull up market calendar: 
exchange_code = list(earnings_df['exchange'].map(stock_ex_codes))
type(exchange_code)
# exchange_code 


list

In [22]:
# add exchange codes to dataframe - to be able to apply look up function
earnings_df.loc[:,'exchange_codes'] = exchange_code
earnings_df

/var/folders/wl/g3zsz07j19b8ryhr2_wvq_c00000gn/T/ipykernel_88176/1640348486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  earnings_df.loc[:,'exchange_codes'] = exchange_code


,date,exchange,symbol,when,security_name,exchange_codes
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDS Corporation - Common Stock,NASDAQ
1,2023-01-03,NASDAQ,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares",NASDAQ
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. - Common Stock",NASDAQ
3,2023-01-04,NASDAQ,SLP,postmarket,"Simulations Plus, Inc. - Common Stock",NASDAQ
4,2023-01-05,NYSE,CAG,premarket,"ConAgra Brands, Inc. Common Stock",XNYS
...,...,...,...,...,...,...
9470,2023-12-21,NYSE,KMX,premarket,CarMax Inc,XNYS
9471,2023-12-21,NASDAQ,LMNR,postmarket,Limoneira Co - Common Stock,NASDAQ
9472,2023-12-21,NYSE,NKE,premarket,"Nike, Inc. Common Stock",XNYS
9473,2023-12-21,NASDAQ,PAYX,premarket,"Paychex, Inc. - Common Stock",NASDAQ


In [23]:
earnings_df['exchange'].value_counts()

exchange
NYSE      4691
NASDAQ    3698
AMEX        67
Name: count, dtype: int64

In [24]:
earnings_df[earnings_df['exchange'].isna()]

,date,exchange,symbol,when,security_name,exchange_codes
10,2023-01-05,NaN,SI,premarket,NaN,NaN
17,2023-01-10,NaN,BBBY,premarket,NaN,NaN
24,2023-01-13,NaN,FRC,premarket,NaN,NaN
63,2023-01-19,NaN,SIVB,postmarket,NaN,NaN
106,2023-01-24,NaN,NXGN,postmarket,NaN,NaN
...,...,...,...,...,...,...
9016,2023-11-09,NaN,LGF,postmarket,NaN,NaN
9148,2023-11-13,NaN,SALM,postmarket,NaN,NaN
9305,2023-11-20,NaN,MARK,premarket,NaN,NaN
9351,2023-11-29,NaN,FTCH,premarket,NaN,NaN


In [25]:
#drop rows that didn't have a trading day that was found:
earnings_df = earnings_df.dropna(inplace = False, ignore_index = True, subset=['exchange', 'security_name', 'exchange_codes']).copy()
earnings_df

,date,exchange,symbol,when,security_name,exchange_codes
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDS Corporation - Common Stock,NASDAQ
1,2023-01-03,NASDAQ,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares",NASDAQ
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. - Common Stock",NASDAQ
3,2023-01-04,NASDAQ,SLP,postmarket,"Simulations Plus, Inc. - Common Stock",NASDAQ
4,2023-01-05,NYSE,CAG,premarket,"ConAgra Brands, Inc. Common Stock",XNYS
...,...,...,...,...,...,...
8451,2023-12-21,NYSE,KMX,premarket,CarMax Inc,XNYS
8452,2023-12-21,NASDAQ,LMNR,postmarket,Limoneira Co - Common Stock,NASDAQ
8453,2023-12-21,NYSE,NKE,premarket,"Nike, Inc. Common Stock",XNYS
8454,2023-12-21,NASDAQ,PAYX,premarket,"Paychex, Inc. - Common Stock",NASDAQ


In [26]:
# store calendars to increase speed - function + apply was 15 mins
nyse_cal = mcal.get_calendar('XNYS')
nasdaq_cal = mcal.get_calendar('NASDAQ')


In [27]:
def get_next_trading_day(row):
    ''' Returns the next trading day that a specific exchange is open based on a input date, considering holidays / weekends.
    If no value for exchange return np.nan. 
    will be applied to a df.
    '''
    next_trading_day2 = np.nan
    try:
        if row['exchange_codes'] == 'XNYS':
            next_trading_day2 = nyse_cal.valid_days(start_date=row['date'], end_date = row['date'] +pd.Timedelta(days=10))[1]
        elif row['exchange_codes'] == 'NASDAQ':
            next_trading_day2 = nasdaq_cal.valid_days(start_date=row['date'], end_date = row['date'] +pd.Timedelta(days=10))[1]
        # exchange =  mcal.get_calendar(row['exchange'])
        # next_trading_day2 = exchange.valid_days(start_date=row['date'], end_date = row['date'] +pd.Timedelta(days=10))[1]
        return next_trading_day2
    except Exception as e:
        print(f'Error:{e}')
        return np.nan

In [28]:
#show how long this cell took to run.
import timeit

start_time = timeit.default_timer()

earnings_df.loc[:,'next_trading_day'] = earnings_df.apply(get_next_trading_day, axis = 1)

elapsed = timeit.default_timer() - start_time
print(f"Time taken: {elapsed} seconds")

Time taken: 1.4507119169999996 seconds


In [29]:
earnings_df

,date,exchange,symbol,when,security_name,exchange_codes,next_trading_day
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDS Corporation - Common Stock,NASDAQ,2023-01-04 00:00:00+00:00
1,2023-01-03,NASDAQ,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares",NASDAQ,2023-01-04 00:00:00+00:00
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. - Common Stock",NASDAQ,2023-01-05 00:00:00+00:00
3,2023-01-04,NASDAQ,SLP,postmarket,"Simulations Plus, Inc. - Common Stock",NASDAQ,2023-01-05 00:00:00+00:00
4,2023-01-05,NYSE,CAG,premarket,"ConAgra Brands, Inc. Common Stock",XNYS,2023-01-06 00:00:00+00:00
...,...,...,...,...,...,...,...
8451,2023-12-21,NYSE,KMX,premarket,CarMax Inc,XNYS,2023-12-22 00:00:00+00:00
8452,2023-12-21,NASDAQ,LMNR,postmarket,Limoneira Co - Common Stock,NASDAQ,2023-12-22 00:00:00+00:00
8453,2023-12-21,NYSE,NKE,premarket,"Nike, Inc. Common Stock",XNYS,2023-12-22 00:00:00+00:00
8454,2023-12-21,NASDAQ,PAYX,premarket,"Paychex, Inc. - Common Stock",NASDAQ,2023-12-22 00:00:00+00:00


In [30]:
earnings_df[earnings_df['next_trading_day'].isnull()]

,date,exchange,symbol,when,security_name,exchange_codes,next_trading_day


In [31]:
#drop rows that didn't have a trading day that was found (there should be none)
earnings_df = earnings_df.dropna(inplace = False, ignore_index = True).copy()
earnings_df

,date,exchange,symbol,when,security_name,exchange_codes,next_trading_day
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDS Corporation - Common Stock,NASDAQ,2023-01-04 00:00:00+00:00
1,2023-01-03,NASDAQ,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares",NASDAQ,2023-01-04 00:00:00+00:00
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. - Common Stock",NASDAQ,2023-01-05 00:00:00+00:00
3,2023-01-04,NASDAQ,SLP,postmarket,"Simulations Plus, Inc. - Common Stock",NASDAQ,2023-01-05 00:00:00+00:00
4,2023-01-05,NYSE,CAG,premarket,"ConAgra Brands, Inc. Common Stock",XNYS,2023-01-06 00:00:00+00:00
...,...,...,...,...,...,...,...
8451,2023-12-21,NYSE,KMX,premarket,CarMax Inc,XNYS,2023-12-22 00:00:00+00:00
8452,2023-12-21,NASDAQ,LMNR,postmarket,Limoneira Co - Common Stock,NASDAQ,2023-12-22 00:00:00+00:00
8453,2023-12-21,NYSE,NKE,premarket,"Nike, Inc. Common Stock",XNYS,2023-12-22 00:00:00+00:00
8454,2023-12-21,NASDAQ,PAYX,premarket,"Paychex, Inc. - Common Stock",NASDAQ,2023-12-22 00:00:00+00:00


In [32]:
# store as datetime types
earnings_df['next_trading_day'] = pd.to_datetime(earnings_df['next_trading_day'])
earnings_df['date'] = pd.to_datetime(earnings_df['date'])
earnings_df.dtypes

date                     datetime64[ns]
exchange                         object
symbol                           object
when                             object
security_name                    object
exchange_codes                   object
next_trading_day    datetime64[ns, UTC]
dtype: object

### Trading Day Before Earnings release

In [37]:
def get_prior_trading_day(row):
    ''' Returns the trading day PRIOR that a specific exchange is open based on a input date, considering holidays / weekends.
    If no value for exchange return np.nan. 
    will be applied to a df.
    start date is 10 days before known trading date, then take second last item from list (last item is the end date) 
    10 days to make sure there's enough range for winter holidays
    '''
    try:
        if row['exchange_codes'] == 'XNYS':
            prior_trading_day = nyse_cal.valid_days(start_date = row['date'] -pd.Timedelta(days=10), end_date=row['date'])[-2]
        elif row['exchange_codes'] == 'NASDAQ':
            prior_trading_day = nasdaq_cal.valid_days(start_date = row['date'] -pd.Timedelta(days=10), end_date=row['date'])[-2]
        return prior_trading_day
        # exchange =  mcal.get_calendar(row['exchange_codes'])
        # prior_trading_day = exchange.valid_days(start_date = row['date'] -pd.Timedelta(days=10), end_date=row['date'])[-2]
        # return prior_trading_day
    except Exception as e:
        # print(f'Error:{e}')
        return np.nan

In [38]:
#show how long this cell took to run.
import timeit

start_time = timeit.default_timer()

earnings_df['prior_trading_day'] = earnings_df.apply(get_prior_trading_day, axis = 1)

elapsed = (timeit.default_timer() - start_time) / 60
print(f"Time taken: {elapsed} minutes")

Time taken: 0.02133772568333335 minutes


In [40]:
earnings_df

,date,exchange,symbol,when,security_name,exchange_codes,next_trading_day,prior_trading_day
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDS Corporation - Common Stock,NASDAQ,2023-01-04 00:00:00+00:00,2022-12-30 00:00:00+00:00
1,2023-01-03,NASDAQ,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares",NASDAQ,2023-01-04 00:00:00+00:00,2022-12-30 00:00:00+00:00
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. - Common Stock",NASDAQ,2023-01-05 00:00:00+00:00,2023-01-03 00:00:00+00:00
3,2023-01-04,NASDAQ,SLP,postmarket,"Simulations Plus, Inc. - Common Stock",NASDAQ,2023-01-05 00:00:00+00:00,2023-01-03 00:00:00+00:00
4,2023-01-05,NYSE,CAG,premarket,"ConAgra Brands, Inc. Common Stock",XNYS,2023-01-06 00:00:00+00:00,2023-01-04 00:00:00+00:00
...,...,...,...,...,...,...,...,...
8451,2023-12-21,NYSE,KMX,premarket,CarMax Inc,XNYS,2023-12-22 00:00:00+00:00,2023-12-20 00:00:00+00:00
8452,2023-12-21,NASDAQ,LMNR,postmarket,Limoneira Co - Common Stock,NASDAQ,2023-12-22 00:00:00+00:00,2023-12-20 00:00:00+00:00
8453,2023-12-21,NYSE,NKE,premarket,"Nike, Inc. Common Stock",XNYS,2023-12-22 00:00:00+00:00,2023-12-20 00:00:00+00:00
8454,2023-12-21,NASDAQ,PAYX,premarket,"Paychex, Inc. - Common Stock",NASDAQ,2023-12-22 00:00:00+00:00,2023-12-20 00:00:00+00:00


### Next trading day or day of 

In [41]:
# QA Confirm not NaN in When
earnings_df['when'].value_counts(dropna=False)

when
premarket     4754
postmarket    3702
Name: count, dtype: int64

In [42]:
## ASSUMPTION - IF NO specification on when, assuming pre market and using same day as release as the eps day (per direction from manager)

def eps_day_after(row):
    eps_day_value = np.nan
    if row['when'] == 'premarket':
        eps_day_value = row['date']
    elif row['when'] == 'postmarket':
         eps_day_value = row['next_trading_day']
    elif pd.isna(row['when']):
        eps_day_value = row['date']
    return eps_day_value

In [43]:
## Based on when the EPS is announced, determine the day we are interested in for movement
earnings_df['after_eps_date'] = earnings_df.apply(eps_day_after, axis = 1)

In [44]:
# Define a function to conditionally replace values
def eps_day_before(row):
    if row['when'] == 'postmarket':
        return row['date']
    elif row['when'] == 'premarket':
        return row['prior_trading_day']
    else:
        return row['next_trading_day']

# Apply the function to create a new column 'D' with the replaced values
earnings_df['before_eps_date'] = earnings_df.apply(eps_day_before, axis=1)


In [45]:
earnings_df['after_eps_date'] = pd.to_datetime(earnings_df['after_eps_date'], utc=True).dt.strftime('%Y-%m-%d')
earnings_df['before_eps_date'] = pd.to_datetime(earnings_df['before_eps_date'], utc=True).dt.strftime('%Y-%m-%d')

In [46]:
earnings_df.head(15)

,date,exchange,symbol,when,security_name,exchange_codes,next_trading_day,prior_trading_day,after_eps_date,before_eps_date
0,2023-01-03,NASDAQ,LEDS,premarket,SemiLEDS Corporation - Common Stock,NASDAQ,2023-01-04 00:00:00+00:00,2022-12-30 00:00:00+00:00,2023-01-03,2022-12-30
1,2023-01-03,NASDAQ,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares",NASDAQ,2023-01-04 00:00:00+00:00,2022-12-30 00:00:00+00:00,2023-01-04,2023-01-03
2,2023-01-04,NASDAQ,RGP,postmarket,"Resources Connection, Inc. - Common Stock",NASDAQ,2023-01-05 00:00:00+00:00,2023-01-03 00:00:00+00:00,2023-01-05,2023-01-04
3,2023-01-04,NASDAQ,SLP,postmarket,"Simulations Plus, Inc. - Common Stock",NASDAQ,2023-01-05 00:00:00+00:00,2023-01-03 00:00:00+00:00,2023-01-05,2023-01-04
4,2023-01-05,NYSE,CAG,premarket,"ConAgra Brands, Inc. Common Stock",XNYS,2023-01-06 00:00:00+00:00,2023-01-04 00:00:00+00:00,2023-01-05,2023-01-04
5,2023-01-05,NASDAQ,HELE,premarket,Helen of Troy Limited - Common Stock,NASDAQ,2023-01-06 00:00:00+00:00,2023-01-04 00:00:00+00:00,2023-01-05,2023-01-04
6,2023-01-05,NYSE,LNN,premarket,Lindsay Corporation Common Stock,XNYS,2023-01-06 00:00:00+00:00,2023-01-04 00:00:00+00:00,2023-01-05,2023-01-04
7,2023-01-05,NYSE,LW,premarket,"Lamb Weston Holdings, Inc. Common Stock",XNYS,2023-01-06 00:00:00+00:00,2023-01-04 00:00:00+00:00,2023-01-05,2023-01-04
8,2023-01-05,NYSE,MSM,premarket,"MSC Industrial Direct Company, Inc. Common Stock",XNYS,2023-01-06 00:00:00+00:00,2023-01-04 00:00:00+00:00,2023-01-05,2023-01-04
9,2023-01-05,NYSE,RPM,premarket,RPM International Inc. Common Stock,XNYS,2023-01-06 00:00:00+00:00,2023-01-04 00:00:00+00:00,2023-01-05,2023-01-04


In [47]:
# tidy up final data set, drop unecessary columns, reorder
earnings_df  = earnings_df.drop(columns = ['next_trading_day', 'prior_trading_day', 'exchange'], inplace=False).copy()

earnings_df = earnings_df.reindex(columns = ['date', 'symbol', 'when', 'security_name', 'exchange_codes', 'before_eps_date', 'after_eps_date'])
earnings_df

,date,symbol,when,security_name,exchange_codes,before_eps_date,after_eps_date
0,2023-01-03,LEDS,premarket,SemiLEDS Corporation - Common Stock,NASDAQ,2022-12-30,2023-01-03
1,2023-01-03,SGH,postmarket,"SMART Global Holdings, Inc. - Ordinary Shares",NASDAQ,2023-01-03,2023-01-04
2,2023-01-04,RGP,postmarket,"Resources Connection, Inc. - Common Stock",NASDAQ,2023-01-04,2023-01-05
3,2023-01-04,SLP,postmarket,"Simulations Plus, Inc. - Common Stock",NASDAQ,2023-01-04,2023-01-05
4,2023-01-05,CAG,premarket,"ConAgra Brands, Inc. Common Stock",XNYS,2023-01-04,2023-01-05
...,...,...,...,...,...,...,...
8451,2023-12-21,KMX,premarket,CarMax Inc,XNYS,2023-12-20,2023-12-21
8452,2023-12-21,LMNR,postmarket,Limoneira Co - Common Stock,NASDAQ,2023-12-21,2023-12-22
8453,2023-12-21,NKE,premarket,"Nike, Inc. Common Stock",XNYS,2023-12-20,2023-12-21
8454,2023-12-21,PAYX,premarket,"Paychex, Inc. - Common Stock",NASDAQ,2023-12-20,2023-12-21


In [48]:
#export to CSV
earnings_df.to_csv('/users/brigitteasullivan/Documents/0. Data Science/Notebooks/Riipen Project/eps-stock-market-ml-project/data/eps_days.csv', index = False)

# BIN

In [ ]:
earnings_df['next_trading_day'] = 

In [ ]:
date_check = earnings_df['date'][10]
stock_check = earnings_df['exchange'][10]
print(date_check)a
print(stock_check

In [ ]:
# next_trading_day = nyse_exchange.valid_days(start_date=date_check, end_date=(date_check + pd.Timedelta(days=10)))[1]
# print('next trading day test: ', next_trading_day)

function_test = get_next_trading_day(date_check, stock_check)
print('function test', function_test)

In [ ]:
    if stock_exchange == "NASDAQ":
        # laksdj
        next_trading_day1 = nasdaq_exchange.valid_days(start_date=date, end_date=date + pd.Timedelta(days=10))[1]
    elif stock_exchange == "NYSE":
        next_trading_day1 = nyse_exchange.valid_days(start_date=date, end_date=date + pd.Timedelta(days=10))[1]
    return next_trading_day1

In [ ]:
earnings_df.head(11)

#### Earnings data

In [ ]:
df = pd.read_csv('EarningsStraddles_RecentEarnings_Results_20240211_210820 copy.csv')



In [ ]:
# % change from close value day prior.  
# Date format = MM/DD/YYYY
# date of earnings release
# Current Price : PRICE of stock on Date

# IMPORTANT:  STOCK VOL and AVG OPTION VOLUME, Symbol	Name	Current Price	% Change	Avg Stock Volume	Avg Option Volume	Date

## NEED Volatility (max / min) for 1 year

In [ ]:
# df['Date'

In [ ]:
df['Name'].value_counts()

In [ ]:
df

### Straddles - Loading

In [ ]:
straddles_df = pd.read_csv('EarningsStraddles_RecentEarnings_Results_20240211_210820 copy.csv')
straddles_df

In [ ]:
straddles_df.sort_values(by='MarketCap', ascending = False)

#### Straddles - cleaning

In [ ]:
# check for outliers

# Create histogram
fig = px.histogram(x=straddles_df['Current Price'])

# Show plot
fig.show()

# remove outliers



In [ ]:
sns.boxplot(straddles_df['Current Price'])

In [ ]:
straddles_df['MarketCap'].value_counts(dropna=False)
straddles_df.dtypes

In [ ]:
# Function to remove outliers based on IQR
def remove_outliers_iqr(df):
    numeric_cols = df.select_dtypes(include=['number']).columns
    Q1 = df[numeric_cols].quantile(0.25)
    Q3 = df[numeric_cols].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[~((df[numeric_cols] < lower_bound) | (df[numeric_cols] > upper_bound)).any(axis=1)]

# Remove outliers from the DataFrame
df_cleaned = remove_outliers_iqr(straddles_df)
df_cleaned

In [ ]:
#drop in watchlist column all values are "0"
df_cleaned['InWatchlist'].value_counts(dropna = False)

In [ ]:
df_cleaned = df_cleaned.copy()
df_cleaned = df_cleaned.drop(columns = ['InWatchlist'])
df_cleaned

In [ ]:
# export cleaned straddles to csv 
df_cleaned.to_csv('straddle_data_cleaned.csv', index = False)


In [ ]:
earnings_df[['when']].value_counts()

In [ ]:
earnings_df.dtypes

In [ ]:
earnings_df['when'].value_counts(dropna=False)

In [ ]:
## export df to csv
earnings_df.to_csv("/users/brigitteasullivan/Documents/0. Data Science/Notebooks/Riipen Project/eps-stock-market-ml-project/data/next_trading_days.csv", index=False)
